
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'


**Import libraries**

In [0]:
import os
import logging
import copy 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
from torch.autograd import Function

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import time
import random



**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 101 # 101 + 1: There is am extra Background class that should be removed 

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 10       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

NUM_CLASSES = 4      # Network output

RANDOM_SEARCH = True # serve per caricare i pesi nella rete senza fare di nuovo il training con i nuovi parametri


**Define Data Preprocessing**

In [0]:
#ImageNet values
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# Define transforms for the train phase
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize( mean, std )                                    
])

# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize( mean, std )                                    
])

**Prepare Dataset**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework3-PACS'):
  !git clone https://github.com/MachineLearning2020/Homework3-PACS.git

DATA_DIR = 'Homework3-PACS/PACS/'

# Prepare Pytorch train/test Datasets
photo_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"photo", transform=train_transform)
art_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"art_painting", transform=eval_transform)
cartoon_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"cartoon", transform=eval_transform)
sketch_dataset = torchvision.datasets.ImageFolder(DATA_DIR+"sketch", transform=eval_transform)

# Check dataset sizes
print('Photo Dataset: {}'.format(len(photo_dataset)))
print('Art Dataset: {}'.format(len(art_dataset)))
print('Cartoon Dataset: {}'.format(len(art_dataset)))
print('Sketch Dataset: {}'.format(len(sketch_dataset)))

**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)

photo_dataloader = DataLoader(photo_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

art_dataloader = DataLoader(art_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,drop_last=True)

cartoon_dataloader = DataLoader(cartoon_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,drop_last=True)

sketch_dataloader = DataLoader(sketch_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,drop_last=True)


Network multi branch

In [0]:
class ConcatLayer(Function):
    @staticmethod
    def forward(ctx, fc6_1 ,fc6_2, fc6_3 ,fc6_4):
        concatenation = torch.stack([fc6_1, fc6_2,fc6_3,fc6_4], dim=0) # esempio (3,4) (3,4)  ---> ( 2, 3, 4 )  oppure ( 4096 )( 4096 ) --> (2,4096)
        #return concatenation.view_as(concatenation)
        return concatenation
        
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output


class OOONet(nn.Module):

    def __init__(self, num_classes=1000):
        super(OOONet, self).__init__()
        
        self.branch1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),                   #FC 6 
            nn.ReLU(inplace=True),
           
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),                   #FC 6 
            nn.ReLU(inplace=True),
           
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),                   #FC 6 
            nn.ReLU(inplace=True),
           
        )
        self.branch4 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  #CONV 5 
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),                   #FC 6 
            nn.ReLU(inplace=True),
           
        )        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
     
        #Livelli di fusione!!!!!!!
        self.concatLayer = ConcatLayer()

        self.classifier = nn.Sequential(
            """
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),                   #FC 6 
            nn.ReLU(inplace=True),
            """
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, NUM_CLASSES ),
        )
        
    
    def forward(self, x):
        #bisogna definire X perchè dovrebbe essere un vettore di 4 immagini.
        x0 = self.features(x[0])
        x0 = self.avgpool(x0)
        output_branch1 = torch.flatten(x0, 1)

        x1 = self.features(x[1])
        x1 = self.avgpool(x1)
        output_branch2 = torch.flatten(x1, 1)

        x2 = self.features(x[2])
        x2 = self.avgpool(x2)
        output_branch3 = torch.flatten(x2, 1)

        x3 = self.features(x[3])
        x3 = self.avgpool(x3)
        output_branch4 = torch.flatten(x3, 1)

        x = self.concatLayer.apply(output_branch1,output_branch2,output_branch3,output_branch4) 
        x = self.classifier(x)

        return x

def buildO3Net ():

    model = alexnet(pretrained=True)

    net =  OOONet()
    #0,3,6,8,10,14

    #DEEP COPY FEATURES OF BRANCH 1

    net.branch1[0].weight.data = copy.deepcopy(model.features[0].weight.data)
    net.branch1[0].bias.data = copy.deepcopy(model.features[0].bias.data)   
    net.branch1[3].weight.data = copy.deepcopy(model.features[3].weight.data)
    net.branch1[3].bias.data = copy.deepcopy(model.features[3].bias.data)
    net.branch1[6].weight.data = copy.deepcopy(model.features[6].weight.data)
    net.branch1[6].bias.data = copy.deepcopy(model.features[6].bias.data)
    net.branch1[8].weight.data = copy.deepcopy(model.features[8].weight.data)
    net.branch1[8].bias.data = copy.deepcopy(model.features[8].bias.data)
    net.branch1[10].weight.data = copy.deepcopy(model.features[10].weight.data)
    net.branch1[10].bias.data = copy.deepcopy(model.features[10].bias.data)
    net.branch1[14].weight.data = copy.deepcopy(model.classifier[1].weight.data)
    net.branch1[14].bias.data = copy.deepcopy(model.classifier[1].bias.data)


    #DEEP COPY FEATURES OF BRANCH 2

    net.branch2[0].weight.data = copy.deepcopy(model.features[0].weight.data)
    net.branch2[0].bias.data = copy.deepcopy(model.features[0].bias.data)   
    net.branch2[3].weight.data = copy.deepcopy(model.features[3].weight.data)
    net.branch2[3].bias.data = copy.deepcopy(model.features[3].bias.data)
    net.branch2[6].weight.data = copy.deepcopy(model.features[6].weight.data)
    net.branch2[6].bias.data = copy.deepcopy(model.features[6].bias.data)
    net.branch2[8].weight.data = copy.deepcopy(model.features[8].weight.data)
    net.branch2[8].bias.data = copy.deepcopy(model.features[8].bias.data)
    net.branch2[10].weight.data = copy.deepcopy(model.features[10].weight.data)
    net.branch2[10].bias.data = copy.deepcopy(model.features[10].bias.data)
    net.branch2[14].weight.data = copy.deepcopy(model.classifier[1].weight.data)
    net.branch2[14].bias.data = copy.deepcopy(model.classifier[1].bias.data)

    #DEEP COPY FEATURES OF BRANCH 3

    net.branch3[0].weight.data = copy.deepcopy(model.features[0].weight.data)
    net.branch3[0].bias.data = copy.deepcopy(model.features[0].bias.data)   
    net.branch3[3].weight.data = copy.deepcopy(model.features[3].weight.data)
    net.branch3[3].bias.data = copy.deepcopy(model.features[3].bias.data)
    net.branch3[6].weight.data = copy.deepcopy(model.features[6].weight.data)
    net.branch3[6].bias.data = copy.deepcopy(model.features[6].bias.data)
    net.branch3[8].weight.data = copy.deepcopy(model.features[8].weight.data)
    net.branch3[8].bias.data = copy.deepcopy(model.features[8].bias.data)
    net.branch3[10].weight.data = copy.deepcopy(model.features[10].weight.data)
    net.branch3[10].bias.data = copy.deepcopy(model.features[10].bias.data)
    net.branch3[14].weight.data = copy.deepcopy(model.classifier[1].weight.data)
    net.branch3[14].bias.data = copy.deepcopy(model.classifier[1].bias.data)

    #DEEP COPY FEATURES OF BRANCH 4

    net.branch4[0].weight.data = copy.deepcopy(model.features[0].weight.data)
    net.branch4[0].bias.data = copy.deepcopy(model.features[0].bias.data)   
    net.branch4[3].weight.data = copy.deepcopy(model.features[3].weight.data)
    net.branch4[3].bias.data = copy.deepcopy(model.features[3].bias.data)
    net.branch4[6].weight.data = copy.deepcopy(model.features[6].weight.data)
    net.branch4[6].bias.data = copy.deepcopy(model.features[6].bias.data)
    net.branch4[8].weight.data = copy.deepcopy(model.features[8].weight.data)
    net.branch4[8].bias.data = copy.deepcopy(model.features[8].bias.data)
    net.branch4[10].weight.data = copy.deepcopy(model.features[10].weight.data)
    net.branch4[10].bias.data = copy.deepcopy(model.features[10].bias.data)
    net.branch4[14].weight.data = copy.deepcopy(model.classifier[1].weight.data)
    net.branch4[14].bias.data = copy.deepcopy(model.classifier[1].bias.data)

    #DEEP COPY OF LAST TWO FC LAYERS

    net.classifier[4].weight.data = copy.deepcopy(model.classifier[4].weight.data)
    net.classifier[4].bias.data = copy.deepcopy(model.classifier[4].bias.data)
    net.classifier[6].weight.data = copy.deepcopy(model.classifier[6].weight.data)
    net.classifier[6].bias.data = copy.deepcopy(model.classifier[6].bias.data)

    return net



**Plot Graph Function** <br>
It s a function used for plotting the graph of all the set of hyperparameters



In [0]:
def plot_graph(epoches ,val_data , train_data, lr,alpha,wd,dp ,xlabel ,ylabel):
  
  epoches = range(epoches)
  plt.plot(epoches,val_data,label="val")
  plt.plot(epoches,train_data,label="train")
  plt.title('Hyperparameters - LR={} ALPHA={} WD={} DP={}'.format(lr,alpha,wd,dp))
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.legend()
  plt.show()    



**Shows data examples**

In [0]:
num_img=10

def imshow(inp,ax, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    ax.imshow(inp)

def visualize(net, dataloader, num):
  net.to(DEVICE)
  net.train(False)
  fig = plt.figure(figsize=(15,7))
  images_so_far = 0

  for images,labels in dataloader:
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    outputs = net(images)
    _, preds = torch.max(outputs.data, 1)
    for j in range(images.size()[0]):
     
      images_so_far += 1
      ax = plt.subplot(2, num/2, images_so_far)
      ax.axis('off')
      ax.set_title('predicted: {}\nreal: {}'.format(art_dataset.classes[preds[j]],art_dataset.classes[labels[j]]))
      imshow(images.cpu().data[j],ax)
      if images_so_far == num:
        return
    plt.show()


visualize(net, art_dataloader, num_img)